# binary Logistic Regression

이항로지스틱회귀는 이진분류문제를 풀기위한 모형이며 종속변수 Y는 범주형 변수로 2가지 값 0또는 1만을 가질 수 있습니다. 파라미터를 사용하여 종속변수 y값을 설명한다는 점에서 선형회귀와 비슷하지만 y가 범주형변수라는 점과 분류문제에 사용되는 모델이라는 차이점이 있습니다.

# 가정
로지스틱회귀의 기본 가정은 다음과 같습니다.

`-` 각각의 데이터요소(샘플)는 여러개의 독립변수와 0또는1만 가질수 있는 종속변수로 이루어져 있습니다. <br>
`-` 임의의 m개의 독립변수와 이진 종속변수를 가지는 i번째 샘플을 나타나면 다음과 같습니다.<br>

- $(x_{1,i},x_{2,i},\dots,x_{m,i}),Y_{i}$<br>

`-` 종속변수 $Y_{i}$는 베르누이 분포를 따르는 확률변수입니다.<br>

- $Y_i|x_{1,i},x_{2,i},\dots,x_{m,i} \sim \text{Bernouli}(p_{i})$<br><br>
독립변수의 값이 주어졌을때 확률변수$Y_{i}$가 따르는 조건부확률분포가 베르누이 분포임을 의미합니다. 확률변수가 어떤 분포를 따르고 있음에 집중한 notation입니다.
- $Pr\,(Y_{i} = y|x_{1,i},x_{2,i},\dots,x_{m,i}\,) =
\begin{cases} 
p_i   & \text{if }y=1, \\
1-p_i & \text{if }y=0
\end{cases} = p_i^y(1-p_i)^{1-y}$<br>
확률변수$Y_{i}$가 따르는 확률함수에 대한 구체적인 식을 보여줍니다. 가장 우측에 있는 식은 중간에 있는 식의 간단한 버전입니다.<br>

기댓값은 실험 또는 시행을 무한히 반복했을때 확률변수가 취하는 값의 평균으로 기대되는(expected) 값(value)입니다. 베르누이분포를 따르는 확률변수 $Y_{i}$의 기댓값을 구하면 다음과 같습니다.<br>

- $E\,[Y_i|x_{1,i},x_{2,i},\dots,x_{m,i}]\, = \sum_{i=0}^{i=1} = 1 \times p_{i} + 0 \times (1-p_{i}) = p_{i}$


## 기댓값에 대한 고찰
기댓값은 실험 또는 시행을 무한히 반복했을때 확률변수가 취하는 값의 평균으로(또는 샘플링된 값의 평균) 기대되는 값입니다. 확률변수가 베르누이 분포를 따르는 경우 확률변수에 대한 기댓값과 베르누이분포의 모수가 같은 값을 가집니다. 그러므로,만약에 주어진 샘플데이터로부터 베르누이분포의 모수를 적절히 추정할 수 있다면 모수는 기댓값이므로 주어진 조건하에서 실험 또는 시행을 무한히 반복할 경우 확률변수가 1인사건과 0인사건중 어떤 사건이 더 많이 발생할지 아는것이므로 많이 발생한 사건 또는 분류된 클래스로 종속변수 Y의 값을 결정하는 것은 타당해보입니다.<br>
`-` e.g.<br>
- $E\,[y_i]\, = \hat{p_i}<0.5$ => 무한히 실행했을때 0인 경우가 더 많을 것임 => 관측치를 0으로 예측<br>
- $E\,[y_i]\, = \hat{p_i}>=0.5$ => 무한히 실행했을때 1인 경우가 더 많을 것임 => 관측치를 1로 예측<br>